<a href="https://colab.research.google.com/github/JKDS87/Prediction-of-Product-Sales/blob/main/Prediction_of_Product_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Product Sales
  - Author: Matthew Malueg

## Project Overview

## Load and Inspect Data

## Clean Data

## Exploratory Data Analysis

## Feature Inspection